In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline 
%config IPCompleter.greedy=True

import collections
import io
import math
import tensorflow as tf
from IPython import display
from sklearn import metrics


## Reading data from file

In [6]:
from collections import Counter

### main dataset ###
data = pd.read_csv("20151219.txt", sep = "\t", header = None)
data.columns = ["conn_len", "service", "src_bytes", "dst_bytes", "conn_numb", "Same_srv_rate", "Serror_rate", "Srv_serror_rate", "Dst_host_count", "Dst_host_srv_count", "Dst_host_same_src_port_rate", "Dst_host_serror_rate", "Dst_host_srv_serror_rate", "Conn_state", "IDS_detection", "Malware_detection", "Ashula_detection", "attack_flag", "src_IP", "src_port", "dst_IP", "dst_port", "start_time", "proto"]

print(Counter(data["attack_flag"]))

Counter({-1: 149915, 1: 67713})


## Transform to binary

In [7]:
data.loc[:, "attack_flag"] = data.loc[:,"attack_flag"].replace(-2, -1)
data.loc[:, "attack_flag"] = data.loc[:, "attack_flag"] * -1
data.loc[:, "attack_flag"] = data.loc[:,"attack_flag"].replace(-1, 0)

print(Counter(data["attack_flag"]))

Counter({1: 149915, 0: 67713})


In [8]:
# removing NaNs
data = data.dropna()
data.shape

(217628, 24)

# a bit of feature engineering

## port numbers classification

In [9]:
t1 = 1023  # well-known port numbers
t2 = 49151 # registered ports
t3 = 65535 # client ports

def wk(data_row):
    if (data_row["src_port"] <= t1): 
        value = 1
    elif ((data_row["src_port"] > t1) and (data_row["src_port"] <= t2)):
        value = 0
    elif ((data_row["src_port"] > t2) and (data_row["src_port"] <= t3)):
        value = 0
    return value

def reg(data_row):
    if (data_row["src_port"] <= t1): 
        value = 0
    elif ((data_row["src_port"] > t1) and (data_row["src_port"] <= t2)):
        value = 1
    elif ((data_row["src_port"] > t2) and (data_row["src_port"] <= t3)):
        value = 0
    return value

def cli(data_row):
    if (data_row["src_port"] <= t1): 
        value = 0
    elif ((data_row["src_port"] > t1) and (data_row["src_port"] <= t2)):
        value = 0
    elif ((data_row["src_port"] > t2) and (data_row["src_port"] <= t3)):
        value = 1
    return value

data["well_known_src_pool"] = data.apply(wk, axis=1)
data["registered_src_pool"] = data.apply(reg, axis=1)
data["cli_src_pool"] = data.apply(cli, axis=1)

In [39]:
data["src_port"] = data["src_port"].apply(str)
data["dst_port"] = data["dst_port"].apply(str)

In [40]:
train, validate, test = np.split(data, [int(.7*len(data)), int(.85*len(data))])
train = train.sample(frac=1)
validate = validate.sample(frac=1)
test = test.sample(frac=1)
y_train = train.loc[:, "attack_flag"].values.ravel()
X_train = train.drop(["attack_flag"], axis=1)
y_validate = validate.loc[:, "attack_flag"].values.ravel()
X_validate = validate.drop(["attack_flag"], axis=1)
y_test = test.loc[:, "attack_flag"].values.ravel()
X_test = test.drop(["attack_flag"], axis=1)
print(train.shape)
print(validate.shape)
print(test.shape)

(152339, 27)
(32644, 27)
(32645, 27)


In [41]:
deep_numeric_feat = ["Same_srv_rate", "Serror_rate", "Srv_serror_rate", "Dst_host_count", "Dst_host_srv_count", "Dst_host_same_src_port_rate", "Dst_host_serror_rate", "Dst_host_srv_serror_rate", "well_known_src_pool", "registered_src_pool", "cli_src_pool"]
wide_numeric_feat = ["conn_len",  "src_bytes", "dst_bytes", "conn_numb"]
categorical_feat = ["service", "Conn_state", "src_port", "dst_port", "proto"]

In [42]:
deep_numeric_feat_tf = [tf.feature_column.numeric_column(k) for k in deep_numeric_feat]
wide_numeric_feat_tf = [tf.feature_column.numeric_column(k) for k in wide_numeric_feat]

In [43]:
#data["src_port"].unique().tolist()

In [44]:
src_port_mod = ["src_" + s for s in data["src_port"].unique().tolist()]
src_port = tf.feature_column.categorical_column_with_vocabulary_list(
    'src_port', src_port_mod)

dst_port_mod = ["dst_" + s for s in data["dst_port"].unique().tolist()]
dst_port = tf.feature_column.categorical_column_with_vocabulary_list(
    'dst_port', dst_port_mod)

service = tf.feature_column.categorical_column_with_vocabulary_list(
    'service', data["service"].unique().tolist())

Conn_state = tf.feature_column.categorical_column_with_vocabulary_list(
    'Conn_state', data["Conn_state"].unique().tolist())

proto = tf.feature_column.categorical_column_with_vocabulary_list(
    'proto', data["proto"].unique().tolist())

In [45]:
deep_columns = [
    tf.feature_column.indicator_column(service),
    tf.feature_column.indicator_column(Conn_state),
    tf.feature_column.indicator_column(proto),
    tf.feature_column.embedding_column(src_port, dimension=10),
    tf.feature_column.embedding_column(dst_port, dimension=3),
]

In [46]:
crossed_columns = [
      tf.feature_column.crossed_column(
          ['src_port', 'service'], hash_bucket_size=10000),
      tf.feature_column.crossed_column(
          ['service', 'proto'], hash_bucket_size=10000),
  ]

In [179]:
model = tf.estimator.DNNLinearCombinedClassifier(
    model_dir = "/home/matz/Desktop/ml_ids/NN_model",
    dnn_activation_fn=tf.nn.selu,
    linear_feature_columns = wide_numeric_feat_tf + crossed_columns,
    dnn_feature_columns = deep_columns + deep_numeric_feat_tf + wide_numeric_feat_tf,
    dnn_hidden_units = [450, 250, 150, 100])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/matz/Desktop/ml_ids/NN_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f711382d080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [180]:
feature_set= deep_numeric_feat + wide_numeric_feat + categorical_feat

In [181]:
def get_input_fn(data_set, num_epochs=None, n_batch = 256, shuffle=True):
    return tf.estimator.inputs.pandas_input_fn(
       x=pd.DataFrame({k: data_set[k].values for k in feature_set}),
       y = pd.Series(data_set["attack_flag"].values),
       batch_size=n_batch,   
       num_epochs=num_epochs,
       shuffle=shuffle)

In [182]:
get_input_fn(test, n_batch=test.shape[0], shuffle=False)

<function tensorflow.python.estimator.inputs.pandas_io.pandas_input_fn.<locals>.input_fn()>

In [ ]:
model.train(input_fn=get_input_fn(test, n_batch=256, shuffle=False), steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /home/matz/Desktop/ml_ids/NN_model/model.ckpt.
INFO:tensorflow:loss = 130.22742, step = 1
INFO:tensorflow:global_step/sec: 108.871
INFO:tensorflow:loss = 6.921026, step = 101 (0.919 sec)
INFO:tensorflow:global_step/sec: 133.669
INFO:tensorflow:loss = 16.337284, step = 201 (0.749 sec)
INFO:tensorflow:global_step/sec: 121.725
INFO:tensorflow:loss = 10.442221, step = 301 (0.821 sec)
INFO:tensorflow:global_step/sec: 116.272
INFO:tensorflow:loss = 7.821759, step = 401 (0.861 sec)
INFO:tensorflow:global_step/sec: 115.361
INFO:tensorflow:loss = 6.4918575, step = 501 (0.867 sec)
INFO:tensorflow:global_step/sec: 117.713
INFO:tensorflow:loss = 6.597049, step = 601 (0.849 sec)
INFO:tensorflow:global_step/sec: 119.551
I

In [ ]:
eval_metrics = model.evaluate(input_fn=get_input_fn(validate, n_batch=validate.shape[0], shuffle=False), steps=1)

In [ ]:
for key,value in sorted(eval_metrics.items()):
  print('%s: %s' % (key, value))

In [ ]:
results = []
results = list(model.predict(input_fn=get_input_fn(test, num_epochs=1, n_batch = X_test.shape[0], shuffle=False)))

In [ ]:
y_pred = []
for el in results:
    y_pred.append(el["class_ids"])

In [ ]:
name = "NN"

Acc = {}
F1S = {}
Prec = {}
Rec = {}
FPR = {}

Acc[name] = metrics.accuracy_score(y_test, y_pred)
F1S[name] = metrics.f1_score(y_test, y_pred)
Prec[name] = metrics.precision_score(y_test, y_pred)
Rec[name] = metrics.recall_score(y_test, y_pred)
tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_pred).ravel()
FPR[name] = fp/(fp+tn)

print("{0:2} Accuracy: {1:.5f}, F1-score: {2:.5f}, Precision: {3:.5f}, Recall: {4:.5f}, FPR: {5:.5f}".format(name, Acc[name], F1S[name], Prec[name], Rec[name], FPR[name]))
print("TN: {0:3}; FP: {1:3}; FN: {2:4}; TP: {3:3}\n".format(tn, fp, fn, tp))